Generating a PCST from an STP input file. 
There aren't many PCST test/input files on the web and thus I've simply generated instances myself. The solutions are compared with the solution of PCST Fast as to guarantee that my solutiongs are correct.

In [7]:
import sys, os
sys.path.append(os.path.abspath('..'))
from graphilp.imports import graph_formats
from graphilp.imports import networkx as imp_nx
import networkx as nx
from gurobipy import *
from pcstHeuristicModule_loops import *
import sys, os
import random
from datetime import datetime
from helpers import *
import flowConfig as conf
%load_ext autoreload
%autoreload 2

import numpy as np

filePath = os.path.join(os.getcwd(), 'testInstances', 'tooBig', 'G101.stp')
G, terminals = graph_formats.stp_to_networkx(filePath)
G2 = G.to_undirected()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
prizes = []
costs = []
edges = []

G = nx.DiGraph(G)

root = terminals[0]
# Giving each node a random prize if its a terminal. If its a steiner Node, set its prize to 0.
for node in G2.nodes():
    if node in terminals:
        thisPrize = random.randint(10000, 100000)
        prizes.append(thisPrize)
        G.nodes[node]['prize'] = thisPrize
    else:
        prizes.append(0)
        G.nodes[node]['prize'] = 0

# Set each edges capacity
for edge in G.edges():
    G[edge[0]][edge[1]]['capacity'] = conf.capacity

# create reverse edge for every edge in the graph
for edge in G.edges(data=True):
    if ((edge[1], edge[0]) in G.edges()):
        continue
    else:
        G.add_edge(edge[1], edge[0], weight = edge[2]['weight'], capacity = edge[2]['capacity'], prize = edge[2]['prize'])
        
sum_of_prizes = sum(prizes)

# Generating input data for PCST Fast
for edge in G2.edges(data=True):
    costs.append(edge[2]['weight'])
    edges.append([edge[0] - 1,edge[1] - 1])
    
prizesNP = np.array(prizes, dtype = float)
costsNP = np.array(costs, dtype = float)
edgesNP = np.array(edges, dtype = int)

In [9]:
# Setting a node as a root. 
Graph = imp_nx.read(G)
root = 69

In [10]:
# Compute the Linear Relaxation of the Model
solutionEdges, solutionNodes, result = computeLP(G, Graph, terminals)

In [11]:
# Change this depending on whether you want to classical solution with for loops or the one 
# with pandas DataFrames. The Pandas DataFrames doesn't work yet. Thus, only use the second method
# for test purposes only!
startTime = datetime.now()
z, y, f = networkConstruction(G, terminals, solutionEdges = solutionEdges, \
                              solutionNodes = solutionNodes, root = root)
# pd_edges, pd_nodes = networkConstruction(G, terminals, solutionEdges = solutionEdges, \
#                                           solutionNodes = solutionNodes, root = root)
print(datetime.now() - startTime)

In [6]:
result, timeElapsed = networkImprovement(G, z, y, f, terminals, root)

<class 'networkx.classes.digraph.DiGraph'>
799914
0
<class 'networkx.classes.digraph.DiGraph'>
799914
1
<class 'networkx.classes.digraph.DiGraph'>
799914
2
<class 'networkx.classes.digraph.DiGraph'>
801237
0
<class 'networkx.classes.digraph.DiGraph'>
801237
1
<class 'networkx.classes.digraph.DiGraph'>
801237
2
<class 'networkx.classes.digraph.DiGraph'>
801237
3
<class 'networkx.classes.digraph.DiGraph'>
947159
0
<class 'networkx.classes.digraph.DiGraph'>
947159
1
<class 'networkx.classes.digraph.DiGraph'>
947159
2
<class 'networkx.classes.digraph.DiGraph'>
947159
3
<class 'networkx.classes.digraph.DiGraph'>
958067
0
<class 'networkx.classes.digraph.DiGraph'>
958067
1
<class 'networkx.classes.digraph.DiGraph'>
958067
2
<class 'networkx.classes.digraph.DiGraph'>
958067
3
<class 'networkx.classes.digraph.DiGraph'>
958067
4
<class 'networkx.classes.digraph.DiGraph'>
965755
0
<class 'networkx.classes.digraph.DiGraph'>
965755
1
<class 'networkx.classes.digraph.DiGraph'>
965755
2
<class 'netw

In [3]:
# Get the PCST Fast solution
vertices, edges = pcst_fast(edgesNP, prizesNP, costsNP, root, 1, 'gw', 1)
# Compute a comparable solution objective
computeMinimizationResultPCSTFast(sum_of_prizes, G, edges, G2.edges)

NameError: name 'pcst_fast' is not defined